In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import pacf
from scipy.stats import norm, chi2

# --- Simulation of a VAR(p) process (example with p=5) ---
np.random.seed(42)
n_obs = 2000
n_vars = 2  # Now we have two variables

# Intercept
intercept = np.array([0.5, -0.2])

# Let's simulate a VAR(5) with 2 variables:
# X_t = intercept + A1*X_{t-1} + A2*X_{t-2} + A3*X_{t-3} + A4*X_{t-4} + A5*X_{t-5} + eps_t
# Choose coefficient matrices that produce a stable process.
A1 = np.array([[0.3, 0.1],
               [0.0, 0.25]])
A2 = np.array([[0.05, 0.0],
               [0.0, 0.05]])
A3 = np.array([[0.02, 0.0],
               [0.0, 0.02]])
A4 = np.array([[0.01, 0.0],
               [0.0, 0.01]])
A5 = np.array([[0.01, 0.0],
               [0.0, 0.01]])

A_mats = [A1, A2, A3, A4, A5]

data = np.zeros((n_obs, n_vars))
eps = np.random.randn(n_obs, n_vars)

for t in range(5, n_obs):
    x_t = intercept.copy()
    for i, A_i in enumerate(A_mats, start=1):
        x_t += data[t - i] @ A_i.T
    x_t += eps[t]
    data[t] = x_t

df = pd.DataFrame(data, columns=[f"var{i}" for i in range(n_vars)])

# --- Select model order using statsmodels ---
maxlags = 10
model = VAR(df)
order_results = model.select_order(maxlags=maxlags)
print("Order Selection Results:")
print(order_results.summary())

aic = order_results.aic
bic = order_results.bic
hqic = order_results.hqic

# Determine the orders that minimize each criterion

morder = np.arange(1, maxlags+1)  # match the dimension of the IC arrays
moaic = morder[np.nanargmin(aic)]
mobic = morder[np.nanargmin(bic)]
mohqc = morder[np.nanargmin(hqic)]


def scale_ic(ic):
    """
    Normalize the information criteria array to the range [0.05, 1].

    Parameters
    ----------
    ic : array-like
        Array of information criterion values (e.g., AIC, BIC, HQIC).

    Returns
    -------
    scaled_ic : array-like
        Normalized values scaled between 0.05 and 1.
    """
    mi, Ma = np.nanmin(ic), np.nanmax(ic)  # Find the minimum and maximum ignoring NaNs
    return (ic - mi) / (Ma - mi) * (1 - 0.05) + 0.05  # Scale to [0.05, 1]

sAIC = scale_ic(aic)
sBIC = scale_ic(bic)
sHQC = scale_ic(hqic)

axes[0].plot(morder, sAIC, 'o-', label=f'AIC (opt={moaic})')
axes[0].plot(morder, sBIC, 'o-', label=f'BIC (opt={mobic})')
axes[0].plot(morder, sHQC, 'o-', label=f'HQC (opt={mohqc})')

moaic = morder[np.nanargmin(aic)]
mobic = morder[np.nanargmin(bic)]
mohqc = morder[np.nanargmin(hqic)]

print(f"AIC best order: {moaic}")
print(f"BIC best order: {mobic}")
print(f"HQC best order: {mohqc}")

# --- Compute LR tests for incrementing model order ---
# Fit models for lag orders 0 to maxlags
LL = []  # Store log-likelihoods
Lm = []  # Effective sample sizes
results_dict = {}
for p in range(maxlags+1):
    if p == 0:
        # AR(0) means no lags, just an intercept
        mu = df.mean().values
        resid = df.values - mu
        sigma = np.cov(resid, rowvar=False)
        # Log-likelihood of a multivariate Gaussian
        n = len(df)
        k = n_vars
        det_sigma = np.linalg.det(sigma)
        ll = -0.5*n*(k*np.log(2*np.pi) + np.log(det_sigma) + 1)
        LL.append(ll)
        Lm.append(n)
        results_dict[p] = None
    else:
        res = model.fit(p)
        LL.append(res.llf)
        # Effective #obs is len(df)-p
        Lm.append(len(df)-p)
        results_dict[p] = res

LL = np.array(LL)
Lm = np.array(Lm)

# For a VAR, each additional lag adds n_vars*n_vars parameters
df_per_lag = n_vars**2

alpha = 0.05
LR_stats = []
LR_pvals = []

# Compare model p vs p-1
for p in range(1, maxlags+1):
    LR = 2*(LL[p] - LL[p-1])
    # Using Chi-square approximation for simplicity:
    # df_params = n_vars*n_vars per extra lag
    df_params = df_per_lag
    LR_p = 1 - chi2.cdf(LR, df_params)
    LR_stats.append(LR)
    LR_pvals.append(LR_p)

LR_stats = np.array(LR_stats)
LR_pvals = np.array(LR_pvals)

# Determine best order based on LR test with Bonferroni correction
lralpha = alpha / maxlags
molrt = 0
for k in range(maxlags, 0, -1):
    if LR_pvals[k-1] < lralpha:
        molrt = k
        break

print(f"LR test best order: {molrt}")

# --- Partial Autocorrelations ---
# We will compute PACF for the first variable "var0" only, as an example.
alpha_pacf = 0.05
pacf_values = pacf(df["var0"], nlags=maxlags)

# Bonferroni correction for pacf significance:
ccalpha = alpha_pacf/maxlags
z = norm.ppf(1-ccalpha/2)
N = len(df)
stderr = 1/np.sqrt(N)
upper_bound = z*stderr
lower_bound = -z*stderr

# --- Plotting ---
fig, axes = plt.subplots(3, 1, figsize=(8, 10))
lags = morder

# 1) Information criteria
def scale_ic(ic):
    mi, Ma = np.nanmin(ic), np.nanmax(ic)
    return (ic - mi)/(Ma - mi)*(1-0.05)+0.05

sAIC = scale_ic(aic)
sBIC = scale_ic(bic)
sHQC = scale_ic(hqic)

axes[0].plot(lags, sAIC, 'o-', label=f'AIC (opt={moaic})')
axes[0].plot(lags, sBIC, 'o-', label=f'BIC (opt={mobic})')
axes[0].plot(lags, sHQC, 'o-', label=f'HQC (opt={mohqc})')
axes[0].set_title("Information Criteria")
axes[0].set_ylabel("Scaled IC")
axes[0].set_xlabel("AR Lags")
axes[0].grid(True)
axes[0].legend()

# 2) LR test p-values
lr_lags = np.arange(1, maxlags+1)
axes[1].semilogy(lr_lags, LR_pvals, 'o-')
axes[1].axhline(y=lralpha, color='r', linestyle='--', label=f'alpha={lralpha:.3g}')
axes[1].set_title(f"Likelihood Ratio Test (alpha={alpha}, Bonferroni corrected)")
axes[1].set_ylabel("p-value (log scale)")
axes[1].set_xlabel("AR Lags")
axes[1].grid(True)
axes[1].legend()

# 3) PACF with significance lines (for var0)
axes[2].plot(lags, pacf_values, 'o')
axes[2].axhline(y=upper_bound, color='r', linestyle='--')
axes[2].axhline(y=lower_bound, color='r', linestyle='--')
axes[2].set_title(f"Partial Autocorrelation (var0) (alpha={alpha_pacf}, Bonferroni corrected)")
axes[2].set_ylabel("PACF")
axes[2].set_xlabel("AR Lags")
axes[2].grid(True)

plt.tight_layout()
plt.show()


Order Selection Results:
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       0.1673      0.1729       1.182      0.1694
1    -0.001446    0.01543*      0.9986    0.004751
2   -0.006331*     0.02179     0.9937*   0.003997*
3    -0.004958     0.03441      0.9951    0.009501
4    -0.003753     0.04686      0.9963     0.01484
5    -0.001282     0.06058      0.9987     0.02144
6   -0.0005146     0.07260      0.9995     0.02634
7    0.0002037     0.08456       1.000     0.03119
8     0.002612     0.09822       1.003     0.03773
9     0.005026      0.1119       1.005     0.04427
10    0.006825      0.1249       1.007     0.05020
--------------------------------------------------


/var/folders/yy/z6qg2k8n2h3g714cbbr9xp040000gn/T/ipykernel_84859/445453403.py:78: RuntimeWarning: invalid value encountered in scalar divide
  return (ic - mi) / (Ma - mi) * (1 - 0.05) + 0.05  # Scale to [0.05, 1]


ValueError: x and y must have same first dimension, but have shapes (10,) and (1,)

In [6]:
df


,var0,var1
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
1995,-0.212884,-1.155883
1996,-1.397559,0.125434
1997,-1.881218,-0.266215
1998,-2.194485,-0.632341


In [7]:
model

In [8]:
order_results